In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Our Import Statements
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import ceil
from pathlib import Path
from seaborn import heatmap
from skimage.transform import rescale
from sklearn.metrics import confusion_matrix

from tensorflow.random import set_seed
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import regularizers
from keras.layers import Dropout

from tensorflow import random
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator


set_seed(4)

def padding(image, shape):
    
    def seperate_pad_width(width):
        return (width // 2, ceil(width / 2))
    
    padding_axis = [seperate_pad_width(abs(image.shape[0] - shape[0])),
                    seperate_pad_width(abs(image.shape[1] - shape[1]))]
    image = np.pad(image, (padding_axis[0], padding_axis[1], (0,0)), constant_values = 0)
    
    return image

def resample(image, shape_read_file, shape_with_padding):
   
    ratio = min(shape_read_file[0] / image.shape[0], shape_read_file[1] / image.shape[1])
    image = rescale(image, ratio, order = 1, preserve_range = True, multichannel = True)
    
    image = padding(image, shape_with_padding) # * shape    
    
    return image

def read_file(path, shape_read_file, shape_with_padding, is_gray=False, is_gonna_be_resampled=True):
    X, Y = [], []
    file = open(path, 'r')
    file.readline() # ignore  header
    
    for line in file:
        temp = line[ :-1].split('#')
        
        image_height = int(temp[3])
        image_width = int(temp[4])
        
        if is_gray:
            image = temp[-1].split(';')
        else:
            image = [(ord(i) & 1) * 255 for i in temp[-1]]
        image = np.array(image, dtype = 'uint8')
        
        image.resize(image_height, image_width, 1)
        if is_gonna_be_resampled:
            image = resample(image, shape_read_file, shape_with_padding)
            
        label = int(temp[0])
        
        X.append(image)
        Y.append(label)
        
    image_count = len(Y)
    if not is_gonna_be_resampled:
        width = max(len(im[0]) for im in X)
        height = max(len(im) for im in X)
        #print(width, height)
        for i in range(len(X)):
            X[i] = resample(X[i], (width, height, 1), (width, height, 1))
    
    X = np.array(X, dtype = 'uint8')
    Y = to_categorical(Y, dtype = 'uint8') # num_classes = 10,  *
    return X,Y

def read_preapare_db(path, shape_read_file, shape_with_padding):
    is_gray = False #'gray' in str(path)
    X, Y = read_file(path, shape_read_file, shape_with_padding, is_gray)
    #X = vpadding(X, shape)
    X = X.astype('float32')
    X /= 255
    return X,Y

def read_preapare_db_original_size(path, shape_read_file=(20,20,1), shape_with_padding=(32,32,1)):
    is_gray = False #'gray' in str(path)
    X, Y = read_file(path, shape_read_file, shape_with_padding, is_gray, is_gonna_be_resampled=False)
    #X = vpadding(X, shape)
    X = X.astype('float32')
    X /= 255
    return X,Y

def read_prepare_db_pdc(path, shape_read_file, shape_with_padding, is_gray=False):
    X, Y = [], []
    file = open(path, 'r')
    file.readline() # ignore  header
    
    for line in file:
        temp = line[ :-1].split('#')
        
        image_height = int(temp[3])
        image_width = int(temp[4])
        
        image = temp[-1].split(';')
        """
        if is_gray:
            image = temp[-1].split(';')
        else:
            image = [(ord(i) & 1) * 255 for i in temp[-1]]
        """
        try:
            image = np.array(image, dtype = 'float32')
        except ValueError as e:
            print(temp[-1])
        
        #image = np.array(image, dtype = 'float32')
        
        #image.resize(image_height, image_width, 1)
        #image = resample(image, shape_read_file, shape_with_padding)
        
        label = int(temp[0])
        
        X.append(image)
        Y.append(label)
        
    #image_count = len(Y)
    X = np.array(X, dtype = 'float32')
    Y = to_categorical(Y, dtype = 'uint8') # num_classes = 10, 
    return X,Y

# 500 Train
#x_train_32x32, y_train = read_preapare_db(Path('datasets/500-100/HandChars_500_woI_Train.txt').absolute(), (20, 20, 1), (32, 32, 1))
#x_train_14x14, _ = read_preapare_db(Path('datasets/500-100/HandChars_500_woI_Train.txt').absolute(), (10, 10, 1), (14, 14, 1))
#x_train_20x20, y_train_20x20_550 = read_preapare_db(Path('datasets/500-100/HandChars_500_woI_Train.txt').absolute(), (20, 20, 1), (20, 20, 1))

# 250 Train
#x_train_32x32, y_train = read_preapare_db(Path('datasets/250/HandChars_250_woI_Train.txt').absolute(), (20, 20, 1), (32, 32, 1))
#x_train_14x14, _ = read_preapare_db(Path('datasets/250/HandChars_250_woI_Train.txt').absolute(), (10, 10, 1), (14, 14, 1))
#x_train_20x20, y_train_20x20_550 = read_preapare_db(Path('datasets/250/HandChars_250_woI_Train.txt').absolute(), (20, 20, 1), (20, 20, 1))


# 500 Train PDC
x_train_pdc, y_train_pdc = read_prepare_db_pdc(Path('datasetsforfinalresults/HandChars32_500_woI_Train_PDC4428.txt').absolute(), (0, 256, 1), (0, 256, 1))

# 250 Train PDC
#x_train_pdc, y_train_pdc = read_prepare_db_pdc(Path('datasets/250/HandChars32_250_woI_Train_PDC4428.txt').absolute(), (0, 256, 1), (0, 256, 1))


# 100 Test
x_test_32x32, y_test = read_preapare_db(Path('datasetsforfinalresults/HandChars_100_Test.txt').absolute(), (20, 20, 1), (32, 32, 1))
x_test_20x20, y_test_20x20 = read_preapare_db(Path('datasetsforfinalresults/HandChars_100_Test.txt').absolute(), (20, 20, 1), (20, 20, 1))
x_test_14x14, _ = read_preapare_db(Path('datasetsforfinalresults/HandChars_100_Test.txt').absolute(), (10, 10, 1), (14, 14, 1))

# 100 Test PDC
x_test_pdc, y_test_pdc = read_prepare_db_pdc(Path('datasetsforfinalresults/HandChars32_100_Test_PDC4428.txt').absolute(), (0, 256, 1), (0, 256, 1))

#x_train_20x20 = x_train_20x20.reshape(x_train_20x20.shape[0], 400)
x_test_20x20 = x_test_20x20.reshape(x_test_20x20.shape[0], 400)

# For all ks

# AUGMENT and PREPARE all datasets

def augment(x_train_original, y_train_original, n, is_shuffled = True):
    datagen = ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
      width_shift_range=0,
      height_shift_range=0,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest')

    """
    horizontal_flip=True,

    """
    it = datagen.flow(x_train_original, y_train_original, batch_size=14000, shuffle=is_shuffled, seed=0) #x_train_32x32 #.reshape(14000, 32, 32, 1)
    x, y = x_train_original, y_train_original #it.next()
    for i in range(n-1):
        temp_x, temp_y = it.next()
        x = np.concatenate((x, temp_x), axis=0)
        y = np.concatenate((y, temp_y), axis=0) #
    #data = np.concatenate((it.next(), it.next()), axis=0)
    #for i in range(n-2):
     #   data = np.concatenate((data, it.next()), axis=0)
    #plt.imshow(resample(data[13999*3], (20, 20, 1), (32, 32, 1)).reshape(32, 32),cmap='Greys')
    return x, y #data

def extract_spec_size(x_train_original, shape_read_file, shape_with_padding):
    x_train_new = list()
    for i in range(len(x_train_original)):
        x_train_new.append(resample(x_train_original[i], shape_read_file, shape_with_padding))
    x_train_new = np.array(x_train_new, dtype = 'float32')
    return x_train_new

# =============== AUGMENT SIZE ===============
#aug_size = 2 #2, 3, 4
# =============== AUGMENT SIZE ===============

# PREPARE 32x32, 20x20 from AUGMENTED

# AUGMENTED Train PDC

# PREPARED PDC files for {seed = 0}
aug_switch = {
    2:'datasetsforfinalresults/GENERATED_HandChars_28k_32x32_Train_PDC4428_saved.txt',
    3:'datasetsforfinalresults/GENERATED_HandChars_42k_32x32_Train_PDC4428_saved.txt',
    5:'datasetsforfinalresults/GENERATED_HandChars_70k_32x32_Train_PDC4428_saved.txt'
}




c:\users\rustem\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\rustem\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
c:\users\rustem\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [ ]:
def conf_2x2(Y_classes, Y_pred_classes, label_map, model_name, seed):
    return
    
# Tezis 1.2

EPOCHS = 200
callback = EarlyStopping(monitor='loss', min_delta=0.01, patience=5)

#models = []
for aug_size in (5, 2, 3): #2, 
    results_df = pd.DataFrame(columns=['Seed', 'Pooling2D', 'Model', 'Param-Count',
                                   'Accuracy-Train', 'Loss-Train', 'Accuracy-Test', 'Loss-Test'])
    print('AUGMENTATION STARTING...')
    x_train_original, y_train_original = read_preapare_db_original_size(Path('datasetsforfinalresults/HandChars_500_woI_Train.txt').absolute())
    print('DATA READ!')
    x_train_original, y_train_original = augment(x_train_original, y_train_original, n=aug_size, is_shuffled = False) # n * 14000
    print('AUGMENTATION DONE', str(x_train_original.shape[0]))
    x_train_pdc, y_train_pdc = read_prepare_db_pdc(Path(aug_switch.get(aug_size, '')).absolute(), (0, 256, 1), (0, 256, 1))
    print('PDC READ')
    y_train = y_train_original
    x_train_32x32 = extract_spec_size(x_train_original, (20, 20, 1), (32, 32, 1))
    x_train_14x14 = extract_spec_size(x_train_original, (10, 10, 1), (14, 14, 1))
    x_train_20x20 = extract_spec_size(x_train_original, (20, 20, 1), (20, 20, 1))
    
    #x_train_new = extract_spec_size(x_train_original, (32, 32, 1), (32, 32, 1)) #(20, 20, 1), (32, 32, 1)
    #x_train_t = x_train_new
    #y_train_t = y_train_new
    #plt.imshow(x_train_new[14000 + 455*28].reshape(32, 32),cmap='Greys')
    #write_to_file(Path('datasets/500-100/GENERATED_HandChars_' + str(n_i*14) + 'k_32x32_Train-18-iyun.txt').absolute(), x_train_new, y_train_new)
    


    x_train_20x20 = x_train_20x20.reshape(x_train_20x20.shape[0], 400)
    print('SIZES DONE!')
    for seed in range(5):
        for Pooling2D in (layers.AveragePooling2D,layers.MaxPooling2D):   #layers.MaxPooling2D, 
            set_seed(seed)

            # conv 1
            keras.backend.clear_session()
            model_LeNet5 = Sequential(name = 'LeNet5')

            model_LeNet5.add(layers.Conv2D(filters=6,  kernel_size=(5,5), padding='valid', activation='relu', input_shape=(32, 32, 1)))
            model_LeNet5.add(Pooling2D(pool_size = (2,2)))

            model_LeNet5.add(layers.Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='relu'))
            model_LeNet5.add(Pooling2D(pool_size=(2,2)))

            model_LeNet5.add(layers.Conv2D(filters=120, kernel_size=(5,5), padding='valid', activation='relu'))
            model_LeNet5.add(layers.Flatten())

            model_LeNet5.add(layers.Dense(84, activation='relu'))
            model_LeNet5.add(layers.Dense(28, activation="softmax"))


            model_LeNet5.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
            model_LeNet5.fit(x_train_32x32, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],
            #kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
            # len(fitted_results.history['loss'])
            loss_train_LeNet5, acc_train_LeNet5 = model_LeNet5.evaluate(x_train_32x32, y_train, verbose=0)
            loss_test_LeNet5, acc_test_LeNet5 = model_LeNet5.evaluate(x_test_32x32, y_test, verbose=0)

            #models.append(model_LeNet5)
            #print(acc_train_LeNet5, '^', acc_test_LeNet5)

            results_df.loc[len(results_df)] = [str(seed), str(Pooling2D)[47:-2], str(model_LeNet5.name),
                                               str(model_LeNet5.count_params()),
                                               acc_train_LeNet5, loss_train_LeNet5,
                                               acc_test_LeNet5, loss_test_LeNet5]
            print(results_df.loc[len(results_df)-1,:].tolist())

            y_pred_classes = np.argmax(model_LeNet5.predict(x_test_32x32), axis = 1) 
            y_classes = np.argmax(y_test, axis = 1)
            conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_LeNet5.name + '-' + str(Pooling2D)[47:-2]
                     , seed)

            # conv 2
            keras.backend.clear_session()
            model_LeNet5 = Sequential(name = 'Conv-14x14')

            model_LeNet5.add(layers.Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='relu', input_shape=(14, 14, 1)))
            model_LeNet5.add(Pooling2D(pool_size=(2,2)))

            model_LeNet5.add(layers.Conv2D(filters=120, kernel_size=(5,5), padding='valid', activation='relu'))
            model_LeNet5.add(layers.Flatten())

            model_LeNet5.add(layers.Dense(84, activation='relu'))
            model_LeNet5.add(layers.Dense(28, activation="softmax"))


            model_LeNet5.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
            model_LeNet5.fit(x_train_14x14, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],


            loss_train_LeNet5, acc_train_LeNet5 = model_LeNet5.evaluate(x_train_14x14, y_train, verbose=0)
            loss_test_LeNet5, acc_test_LeNet5 = model_LeNet5.evaluate(x_test_14x14, y_test, verbose=0)

            results_df.loc[len(results_df)] = [str(seed), str(Pooling2D)[47:-2], str(model_LeNet5.name),
                                               str(model_LeNet5.count_params()),
                                               acc_train_LeNet5, loss_train_LeNet5,
                                               acc_test_LeNet5, loss_test_LeNet5]
            print(results_df.loc[len(results_df)-1,:].tolist())

            y_pred_classes = np.argmax(model_LeNet5.predict(x_test_14x14), axis = 1) 
            y_classes = np.argmax(y_test, axis = 1)
            conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_LeNet5.name + '-' + str(Pooling2D)[47:-2]
                     , seed)

            # conv 3
            keras.backend.clear_session()
            model_LeNet5 = Sequential(name = 'LeNet5-filters_6-8')

            model_LeNet5.add(layers.Conv2D(filters=6,  kernel_size=(5,5), padding='valid', activation='relu', input_shape=(32, 32, 1)))
            model_LeNet5.add(Pooling2D(pool_size = (2,2)))

            model_LeNet5.add(layers.Conv2D(filters=8, kernel_size=(5,5), padding='valid', activation='relu'))
            model_LeNet5.add(Pooling2D(pool_size=(2,2)))

            model_LeNet5.add(layers.Conv2D(filters=120, kernel_size=(5,5), padding='valid', activation='relu'))
            model_LeNet5.add(layers.Flatten())

            model_LeNet5.add(layers.Dense(84, activation='relu'))
            model_LeNet5.add(layers.Dense(28, activation="softmax"))


            model_LeNet5.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
            model_LeNet5.fit(x_train_32x32, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],


            loss_train_LeNet5, acc_train_LeNet5 = model_LeNet5.evaluate(x_train_32x32, y_train, verbose=0)
            loss_test_LeNet5, acc_test_LeNet5 = model_LeNet5.evaluate(x_test_32x32, y_test, verbose=0)

            #print(acc_train_LeNet5, '^', acc_test_LeNet5)

            results_df.loc[len(results_df)] = [str(seed), str(Pooling2D)[47:-2], str(model_LeNet5.name),
                                               str(model_LeNet5.count_params()),
                                               acc_train_LeNet5, loss_train_LeNet5,
                                               acc_test_LeNet5, loss_test_LeNet5]
            print(results_df.loc[len(results_df)-1,:].tolist())

            y_pred_classes = np.argmax(model_LeNet5.predict(x_test_32x32), axis = 1) 
            y_classes = np.argmax(y_test, axis = 1)
            conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_LeNet5.name + '-' + str(Pooling2D)[47:-2]
                     , seed)

            # conv 4
            keras.backend.clear_session()
            model_LeNet5 = Sequential(name = 'LeNet5-filters_6-4')

            model_LeNet5.add(layers.Conv2D(filters=6,  kernel_size=(5,5), padding='valid', activation='relu', input_shape=(32, 32, 1)))
            model_LeNet5.add(Pooling2D(pool_size = (2,2)))

            model_LeNet5.add(layers.Conv2D(filters=4, kernel_size=(5,5), padding='valid', activation='relu'))
            model_LeNet5.add(Pooling2D(pool_size=(2,2)))

            model_LeNet5.add(layers.Conv2D(filters=120, kernel_size=(5,5), padding='valid', activation='relu'))
            model_LeNet5.add(layers.Flatten())

            model_LeNet5.add(layers.Dense(84, activation='relu'))
            model_LeNet5.add(layers.Dense(28, activation="softmax"))


            model_LeNet5.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
            model_LeNet5.fit(x_train_32x32, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],


            loss_train_LeNet5, acc_train_LeNet5 = model_LeNet5.evaluate(x_train_32x32, y_train, verbose=0)
            loss_test_LeNet5, acc_test_LeNet5 = model_LeNet5.evaluate(x_test_32x32, y_test, verbose=0)

            #print(acc_train_LeNet5, '^', acc_test_LeNet5)

            results_df.loc[len(results_df)] = [str(seed), str(Pooling2D)[47:-2], str(model_LeNet5.name),
                                               str(model_LeNet5.count_params()),
                                               acc_train_LeNet5, loss_train_LeNet5,
                                               acc_test_LeNet5, loss_test_LeNet5]
            print(results_df.loc[len(results_df)-1,:].tolist())

            y_pred_classes = np.argmax(model_LeNet5.predict(x_test_32x32), axis = 1) 
            y_classes = np.argmax(y_test, axis = 1)
            conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_LeNet5.name + '-' + str(Pooling2D)[47:-2]
                     , seed)


        # pixels 1
        keras.backend.clear_session()    
        model_pixels_50_50 = Sequential(name = 'Pixels-400-50-50-28')

        model_pixels_50_50.add(layers.Dense(50, input_shape = (400,), activation='relu'))
        model_pixels_50_50.add(layers.Dense(50, activation='relu'))
        model_pixels_50_50.add(layers.Dense(28, activation='softmax'))

        model_pixels_50_50.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
        model_pixels_50_50.fit(x_train_20x20, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],

        loss_train_pixels_50_50, acc_train_pixels_50_50 = model_pixels_50_50.evaluate(x_train_20x20, y_train, verbose=0)
        loss_test_pixels_50_50, acc_test_pixels_50_50 = model_pixels_50_50.evaluate(x_test_20x20, y_test, verbose=0)

        results_df.loc[len(results_df)] = [str(seed), str('-'), str(model_pixels_50_50.name),
                                           str(model_pixels_50_50.count_params()),
                                           acc_train_pixels_50_50, loss_train_pixels_50_50,
                                           acc_test_pixels_50_50, loss_test_pixels_50_50]        
        print(results_df.loc[len(results_df)-1,:].tolist())

        y_pred_classes = np.argmax(model_pixels_50_50.predict(x_test_20x20), axis = 1) 
        y_classes = np.argmax(y_test, axis = 1)
        conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_pixels_50_50.name, seed)

        #pixels 2
        keras.backend.clear_session()    
        model_pixels_50 = Sequential(name = 'Pixels-400-50-28')

        model_pixels_50.add(layers.Dense(50, input_shape = (400,), activation='relu'))
        model_pixels_50.add(layers.Dense(28, activation='softmax'))

        model_pixels_50.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
        model_pixels_50.fit(x_train_20x20, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],

        loss_train_pixels_50_50, acc_train_pixels_50_50 = model_pixels_50.evaluate(x_train_20x20, y_train, verbose=0)
        loss_test_pixels_50_50, acc_test_pixels_50_50 = model_pixels_50.evaluate(x_test_20x20, y_test, verbose=0)

        results_df.loc[len(results_df)] = [str(seed), str('-'), str(model_pixels_50.name),
                                           str(model_pixels_50.count_params()),
                                           acc_train_pixels_50_50, loss_train_pixels_50_50,
                                           acc_test_pixels_50_50, loss_test_pixels_50_50]        
        print(results_df.loc[len(results_df)-1,:].tolist())

        y_pred_classes = np.argmax(model_pixels_50.predict(x_test_20x20), axis = 1) 
        y_classes = np.argmax(y_test, axis = 1)
        conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_pixels_50.name, seed)

        #pdc 1
        keras.backend.clear_session()
        model_pdc_50_50 = Sequential(name = 'PDC-256-50-50-28')

        model_pdc_50_50.add(layers.Dense(50, input_shape = (256,), activation='relu'))
        model_pdc_50_50.add(layers.Dense(50, activation='relu'))
        model_pdc_50_50.add(layers.Dense(28, activation='softmax'))

        model_pdc_50_50.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
        model_pdc_50_50.fit(x_train_pdc, y_train_pdc, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],

        loss_train_pdc_50_50, acc_train_pdc_50_50 = model_pdc_50_50.evaluate(x_train_pdc, y_train_pdc, verbose=0)
        loss_test_pdc_50_50, acc_test_pdc_50_50 = model_pdc_50_50.evaluate(x_test_pdc, y_test_pdc, verbose=0)

        results_df.loc[len(results_df)] = [str(seed), str('-'), str(model_pdc_50_50.name),
                                           str(model_pdc_50_50.count_params()),
                                           acc_train_pdc_50_50, loss_train_pdc_50_50,
                                           acc_test_pdc_50_50, loss_test_pdc_50_50]        
        print(results_df.loc[len(results_df)-1,:].tolist())

        y_pred_classes = np.argmax(model_pdc_50_50.predict(x_test_pdc), axis = 1) 
        y_classes = np.argmax(y_test_pdc, axis = 1)
        conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_pdc_50_50.name, seed)

        #pdc 2
        keras.backend.clear_session()
        model_pdc_50 = Sequential(name = 'PDC-256-50-28')

        model_pdc_50.add(layers.Dense(50, input_shape = (256,), activation='relu'))
        model_pdc_50.add(layers.Dense(28, activation='softmax'))

        model_pdc_50.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
        model_pdc_50.fit(x_train_pdc, y_train_pdc, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],

        loss_train_pdc_50, acc_train_pdc_50 = model_pdc_50.evaluate(x_train_pdc, y_train_pdc, verbose=0)
        loss_test_pdc_50, acc_test_pdc_50 = model_pdc_50.evaluate(x_test_pdc, y_test_pdc, verbose=0)

        results_df.loc[len(results_df)] = [str(seed), str('-'), str(model_pdc_50.name),
                                           str(model_pdc_50.count_params()),
                                           acc_train_pdc_50, loss_train_pdc_50,
                                           acc_test_pdc_50, loss_test_pdc_50]        
        print(results_df.loc[len(results_df)-1,:].tolist())

        y_pred_classes = np.argmax(model_pdc_50.predict(x_test_pdc), axis = 1) 
        y_classes = np.argmax(y_test_pdc, axis = 1)
        conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_pdc_50.name, seed)

        results_df.to_csv(r'Results_train-' + str(aug_size * 14) +'k_all-conv-pixels-pdc-added.csv', index = False)

In [2]:
def conf_2x2(Y_classes, Y_pred_classes, label_map, model_name, seed):
    return
    
# Tezis 1.2

EPOCHS = 200
callback = EarlyStopping(monitor='loss', min_delta=0.01, patience=5)

#models = []
aug_size = 3 
results_df = pd.DataFrame(columns=['Seed', 'Pooling2D', 'Model', 'Param-Count',
                               'Accuracy-Train', 'Loss-Train', 'Accuracy-Test', 'Loss-Test'])
print('AUGMENTATION STARTING...')
x_train_original, y_train_original = read_preapare_db_original_size(Path('datasetsforfinalresults/HandChars_500_woI_Train.txt').absolute())
print('DATA READ!')
x_train_original, y_train_original = augment(x_train_original, y_train_original, n=aug_size, is_shuffled = False) # n * 14000
print('AUGMENTATION DONE', str(x_train_original.shape[0]))
x_train_pdc, y_train_pdc = read_prepare_db_pdc(Path(aug_switch.get(aug_size, '')).absolute(), (0, 256, 1), (0, 256, 1))
print('PDC READ')
y_train = y_train_original
x_train_32x32 = extract_spec_size(x_train_original, (20, 20, 1), (32, 32, 1))
x_train_14x14 = extract_spec_size(x_train_original, (10, 10, 1), (14, 14, 1))
x_train_20x20 = extract_spec_size(x_train_original, (20, 20, 1), (20, 20, 1))

#x_train_new = extract_spec_size(x_train_original, (32, 32, 1), (32, 32, 1)) #(20, 20, 1), (32, 32, 1)
#x_train_t = x_train_new
#y_train_t = y_train_new
#plt.imshow(x_train_new[14000 + 455*28].reshape(32, 32),cmap='Greys')
#write_to_file(Path('datasets/500-100/GENERATED_HandChars_' + str(n_i*14) + 'k_32x32_Train-18-iyun.txt').absolute(), x_train_new, y_train_new)



x_train_20x20 = x_train_20x20.reshape(x_train_20x20.shape[0], 400)
print('SIZES DONE!')
for seed in range(5):
    for Pooling2D in (layers.AveragePooling2D,layers.MaxPooling2D):   #layers.MaxPooling2D, 
        set_seed(seed)
        print('CLEARSESSION STARTED')
        # conv 1
        keras.backend.clear_session()
        print('CLEARSESSION DONE')
        model_LeNet5 = Sequential(name = 'LeNet5')

        model_LeNet5.add(layers.Conv2D(filters=6,  kernel_size=(5,5), padding='valid', activation='relu', input_shape=(32, 32, 1)))
        model_LeNet5.add(Pooling2D(pool_size = (2,2)))

        model_LeNet5.add(layers.Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='relu'))
        model_LeNet5.add(Pooling2D(pool_size=(2,2)))

        model_LeNet5.add(layers.Conv2D(filters=120, kernel_size=(5,5), padding='valid', activation='relu'))
        model_LeNet5.add(layers.Flatten())

        model_LeNet5.add(layers.Dense(84, activation='relu'))
        model_LeNet5.add(layers.Dense(28, activation="softmax"))

        print('MODEL STR DEFINED')
        model_LeNet5.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
        model_LeNet5.fit(x_train_32x32, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],
        #kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
        # len(fitted_results.history['loss'])
        print('FITTED')
        loss_train_LeNet5, acc_train_LeNet5 = model_LeNet5.evaluate(x_train_32x32, y_train, verbose=0)
        loss_test_LeNet5, acc_test_LeNet5 = model_LeNet5.evaluate(x_test_32x32, y_test, verbose=0)
        
        #models.append(model_LeNet5)
        #print(acc_train_LeNet5, '^', acc_test_LeNet5)

        results_df.loc[len(results_df)] = [str(seed), str(Pooling2D)[47:-2], str(model_LeNet5.name),
                                           str(model_LeNet5.count_params()),
                                           acc_train_LeNet5, loss_train_LeNet5,
                                           acc_test_LeNet5, loss_test_LeNet5]
        print(results_df.loc[len(results_df)-1,:].tolist())

        y_pred_classes = np.argmax(model_LeNet5.predict(x_test_32x32), axis = 1) 
        y_classes = np.argmax(y_test, axis = 1)
        conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_LeNet5.name + '-' + str(Pooling2D)[47:-2]
                 , seed)

        # conv 2
        keras.backend.clear_session()
        model_LeNet5 = Sequential(name = 'Conv-14x14')

        model_LeNet5.add(layers.Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='relu', input_shape=(14, 14, 1)))
        model_LeNet5.add(Pooling2D(pool_size=(2,2)))

        model_LeNet5.add(layers.Conv2D(filters=120, kernel_size=(5,5), padding='valid', activation='relu'))
        model_LeNet5.add(layers.Flatten())

        model_LeNet5.add(layers.Dense(84, activation='relu'))
        model_LeNet5.add(layers.Dense(28, activation="softmax"))


        model_LeNet5.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
        model_LeNet5.fit(x_train_14x14, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],


        loss_train_LeNet5, acc_train_LeNet5 = model_LeNet5.evaluate(x_train_14x14, y_train, verbose=0)
        loss_test_LeNet5, acc_test_LeNet5 = model_LeNet5.evaluate(x_test_14x14, y_test, verbose=0)

        results_df.loc[len(results_df)] = [str(seed), str(Pooling2D)[47:-2], str(model_LeNet5.name),
                                           str(model_LeNet5.count_params()),
                                           acc_train_LeNet5, loss_train_LeNet5,
                                           acc_test_LeNet5, loss_test_LeNet5]
        print(results_df.loc[len(results_df)-1,:].tolist())

        y_pred_classes = np.argmax(model_LeNet5.predict(x_test_14x14), axis = 1) 
        y_classes = np.argmax(y_test, axis = 1)
        conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_LeNet5.name + '-' + str(Pooling2D)[47:-2]
                 , seed)

        # conv 3
        keras.backend.clear_session()
        model_LeNet5 = Sequential(name = 'LeNet5-filters_6-8')

        model_LeNet5.add(layers.Conv2D(filters=6,  kernel_size=(5,5), padding='valid', activation='relu', input_shape=(32, 32, 1)))
        model_LeNet5.add(Pooling2D(pool_size = (2,2)))

        model_LeNet5.add(layers.Conv2D(filters=8, kernel_size=(5,5), padding='valid', activation='relu'))
        model_LeNet5.add(Pooling2D(pool_size=(2,2)))

        model_LeNet5.add(layers.Conv2D(filters=120, kernel_size=(5,5), padding='valid', activation='relu'))
        model_LeNet5.add(layers.Flatten())

        model_LeNet5.add(layers.Dense(84, activation='relu'))
        model_LeNet5.add(layers.Dense(28, activation="softmax"))


        model_LeNet5.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
        model_LeNet5.fit(x_train_32x32, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],


        loss_train_LeNet5, acc_train_LeNet5 = model_LeNet5.evaluate(x_train_32x32, y_train, verbose=0)
        loss_test_LeNet5, acc_test_LeNet5 = model_LeNet5.evaluate(x_test_32x32, y_test, verbose=0)

        #print(acc_train_LeNet5, '^', acc_test_LeNet5)

        results_df.loc[len(results_df)] = [str(seed), str(Pooling2D)[47:-2], str(model_LeNet5.name),
                                           str(model_LeNet5.count_params()),
                                           acc_train_LeNet5, loss_train_LeNet5,
                                           acc_test_LeNet5, loss_test_LeNet5]
        print(results_df.loc[len(results_df)-1,:].tolist())

        y_pred_classes = np.argmax(model_LeNet5.predict(x_test_32x32), axis = 1) 
        y_classes = np.argmax(y_test, axis = 1)
        conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_LeNet5.name + '-' + str(Pooling2D)[47:-2]
                 , seed)

        # conv 4
        keras.backend.clear_session()
        model_LeNet5 = Sequential(name = 'LeNet5-filters_6-4')

        model_LeNet5.add(layers.Conv2D(filters=6,  kernel_size=(5,5), padding='valid', activation='relu', input_shape=(32, 32, 1)))
        model_LeNet5.add(Pooling2D(pool_size = (2,2)))

        model_LeNet5.add(layers.Conv2D(filters=4, kernel_size=(5,5), padding='valid', activation='relu'))
        model_LeNet5.add(Pooling2D(pool_size=(2,2)))

        model_LeNet5.add(layers.Conv2D(filters=120, kernel_size=(5,5), padding='valid', activation='relu'))
        model_LeNet5.add(layers.Flatten())

        model_LeNet5.add(layers.Dense(84, activation='relu'))
        model_LeNet5.add(layers.Dense(28, activation="softmax"))


        model_LeNet5.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
        model_LeNet5.fit(x_train_32x32, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],


        loss_train_LeNet5, acc_train_LeNet5 = model_LeNet5.evaluate(x_train_32x32, y_train, verbose=0)
        loss_test_LeNet5, acc_test_LeNet5 = model_LeNet5.evaluate(x_test_32x32, y_test, verbose=0)

        #print(acc_train_LeNet5, '^', acc_test_LeNet5)

        results_df.loc[len(results_df)] = [str(seed), str(Pooling2D)[47:-2], str(model_LeNet5.name),
                                           str(model_LeNet5.count_params()),
                                           acc_train_LeNet5, loss_train_LeNet5,
                                           acc_test_LeNet5, loss_test_LeNet5]
        print(results_df.loc[len(results_df)-1,:].tolist())

        y_pred_classes = np.argmax(model_LeNet5.predict(x_test_32x32), axis = 1) 
        y_classes = np.argmax(y_test, axis = 1)
        conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_LeNet5.name + '-' + str(Pooling2D)[47:-2]
                 , seed)


    # pixels 1
    keras.backend.clear_session()    
    model_pixels_50_50 = Sequential(name = 'Pixels-400-50-50-28')

    model_pixels_50_50.add(layers.Dense(50, input_shape = (400,), activation='relu'))
    model_pixels_50_50.add(layers.Dense(50, activation='relu'))
    model_pixels_50_50.add(layers.Dense(28, activation='softmax'))

    model_pixels_50_50.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
    model_pixels_50_50.fit(x_train_20x20, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],

    loss_train_pixels_50_50, acc_train_pixels_50_50 = model_pixels_50_50.evaluate(x_train_20x20, y_train, verbose=0)
    loss_test_pixels_50_50, acc_test_pixels_50_50 = model_pixels_50_50.evaluate(x_test_20x20, y_test, verbose=0)

    results_df.loc[len(results_df)] = [str(seed), str('-'), str(model_pixels_50_50.name),
                                       str(model_pixels_50_50.count_params()),
                                       acc_train_pixels_50_50, loss_train_pixels_50_50,
                                       acc_test_pixels_50_50, loss_test_pixels_50_50]        
    print(results_df.loc[len(results_df)-1,:].tolist())

    y_pred_classes = np.argmax(model_pixels_50_50.predict(x_test_20x20), axis = 1) 
    y_classes = np.argmax(y_test, axis = 1)
    conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_pixels_50_50.name, seed)

    #pixels 2
    keras.backend.clear_session()    
    model_pixels_50 = Sequential(name = 'Pixels-400-50-28')

    model_pixels_50.add(layers.Dense(50, input_shape = (400,), activation='relu'))
    model_pixels_50.add(layers.Dense(28, activation='softmax'))

    model_pixels_50.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
    model_pixels_50.fit(x_train_20x20, y_train, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],

    loss_train_pixels_50_50, acc_train_pixels_50_50 = model_pixels_50.evaluate(x_train_20x20, y_train, verbose=0)
    loss_test_pixels_50_50, acc_test_pixels_50_50 = model_pixels_50.evaluate(x_test_20x20, y_test, verbose=0)

    results_df.loc[len(results_df)] = [str(seed), str('-'), str(model_pixels_50.name),
                                       str(model_pixels_50.count_params()),
                                       acc_train_pixels_50_50, loss_train_pixels_50_50,
                                       acc_test_pixels_50_50, loss_test_pixels_50_50]        
    print(results_df.loc[len(results_df)-1,:].tolist())

    y_pred_classes = np.argmax(model_pixels_50.predict(x_test_20x20), axis = 1) 
    y_classes = np.argmax(y_test, axis = 1)
    conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_pixels_50.name, seed)

    #pdc 1
    keras.backend.clear_session()
    model_pdc_50_50 = Sequential(name = 'PDC-256-50-50-28')

    model_pdc_50_50.add(layers.Dense(50, input_shape = (256,), activation='relu'))
    model_pdc_50_50.add(layers.Dense(50, activation='relu'))
    model_pdc_50_50.add(layers.Dense(28, activation='softmax'))

    model_pdc_50_50.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
    model_pdc_50_50.fit(x_train_pdc, y_train_pdc, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],

    loss_train_pdc_50_50, acc_train_pdc_50_50 = model_pdc_50_50.evaluate(x_train_pdc, y_train_pdc, verbose=0)
    loss_test_pdc_50_50, acc_test_pdc_50_50 = model_pdc_50_50.evaluate(x_test_pdc, y_test_pdc, verbose=0)

    results_df.loc[len(results_df)] = [str(seed), str('-'), str(model_pdc_50_50.name),
                                       str(model_pdc_50_50.count_params()),
                                       acc_train_pdc_50_50, loss_train_pdc_50_50,
                                       acc_test_pdc_50_50, loss_test_pdc_50_50]        
    print(results_df.loc[len(results_df)-1,:].tolist())

    y_pred_classes = np.argmax(model_pdc_50_50.predict(x_test_pdc), axis = 1) 
    y_classes = np.argmax(y_test_pdc, axis = 1)
    conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_pdc_50_50.name, seed)

    #pdc 2
    keras.backend.clear_session()
    model_pdc_50 = Sequential(name = 'PDC-256-50-28')

    model_pdc_50.add(layers.Dense(50, input_shape = (256,), activation='relu'))
    model_pdc_50.add(layers.Dense(28, activation='softmax'))

    model_pdc_50.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics = ['accuracy'])
    model_pdc_50.fit(x_train_pdc, y_train_pdc, epochs = EPOCHS, shuffle = True, callbacks=[callback], verbose=0) #callbacks=[callback],

    loss_train_pdc_50, acc_train_pdc_50 = model_pdc_50.evaluate(x_train_pdc, y_train_pdc, verbose=0)
    loss_test_pdc_50, acc_test_pdc_50 = model_pdc_50.evaluate(x_test_pdc, y_test_pdc, verbose=0)

    results_df.loc[len(results_df)] = [str(seed), str('-'), str(model_pdc_50.name),
                                       str(model_pdc_50.count_params()),
                                       acc_train_pdc_50, loss_train_pdc_50,
                                       acc_test_pdc_50, loss_test_pdc_50]        
    print(results_df.loc[len(results_df)-1,:].tolist())

    y_pred_classes = np.argmax(model_pdc_50.predict(x_test_pdc), axis = 1) 
    y_classes = np.argmax(y_test_pdc, axis = 1)
    conf_2x2(y_classes, y_pred_classes, 'ABCÇDEƏFGHIJKQLMNOPRSŞTUVXYZ', model_pdc_50.name, seed)

    results_df.to_csv(r'Results_train-' + str(aug_size * 14) +'k_all-conv-pixels-pdc-added.csv', index = False)

AUGMENTATION STARTING...
DATA READ!


C:\Users\Rustem\AppData\Roaming\Python\Python37\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Rustem\AppData\Roaming\Python\Python37\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


AUGMENTATION DONE 42000
PDC READ
SIZES DONE!
CLEARSESSION STARTED
CLEARSESSION DONE
MODEL STR DEFINED
FITTED
['0', 'AveragePooling2D', 'LeNet5', '63236', 0.9913333058357239, 0.02424595132470131, 0.871071457862854, 1.4405299425125122]
['0', 'AveragePooling2D', 'Conv-14x14', '61080', 0.9869285821914673, 0.03507982939481735, 0.783214271068573, 1.9217894077301025]
['0', 'AveragePooling2D', 'LeNet5-filters_6-8', '38028', 0.9913809299468994, 0.02406124956905842, 0.8739285469055176, 1.2112934589385986]
['0', 'AveragePooling2D', 'LeNet5-filters_6-4', '25424', 0.991190493106842, 0.024006469175219536, 0.866428554058075, 1.2757506370544434]
CLEARSESSION STARTED
CLEARSESSION DONE
MODEL STR DEFINED
FITTED
['0', 'MaxPooling2D', 'LeNet5', '63236', 0.9933333396911621, 0.02050248719751835, 0.8739285469055176, 1.3196334838867188]
['0', 'MaxPooling2D', 'Conv-14x14', '61080', 0.9902142882347107, 0.027915742248296738, 0.7814285755157471, 2.15394926071167]
['0', 'MaxPooling2D', 'LeNet5-filters_6-8', '38028'

['4', '-', 'PDC-256-50-28', '14278', 0.9876904487609863, 0.03584042936563492, 0.8732143044471741, 1.2610753774642944]
